# Importing libs

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Bidirectional, LSTM, Dense, Input, Reshape
)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Attention

2024-11-17 12:28:23.839228: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 12:28:23.937871: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 12:28:24.081423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731857304.201808   18180 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731857304.242082   18180 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 12:28:24.517100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Creating Model

In [3]:
def build_crnn_with_attention(input_shape, num_classes):
    timesteps = 5
    # Input
    inputs = Input(shape=input_shape, name="input_layer")

    # Convolutional Layers
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Flatten and Reshape for RNN Input
    x = Reshape((timesteps, -1))(x)

    # Bidirectional LSTM
    x = Bidirectional(LSTM(128, return_sequences=True))(x)

    # Attention Mechanism
    attention_output = Attention()([x, x])

    # Fully Connected Layer for Classification
    x = Dense(num_classes, activation="softmax", name="output_layer")(attention_output)

    outputs = Reshape((timesteps, num_classes))(x)
    # Model
    model = Model(inputs, outputs)
    return model


# Creating Data for training and Testing

In [4]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import LeaveOneOut, KFold


In [5]:
import cv2
import numpy as np
from PIL import Image
import io
import base64

def cv_image_processing(image_path, gaussian_kernel, sigma, median_kernel, closing_k, dilation_k, method):
	kernel_d = np.ones(dilation_k, np.uint8)
	kernel_c = np.ones(closing_k, np.uint8)

	img = cv2.imread(image_path, 0)
	(h, w) = img.shape[:2]
	img = cv2.resize(img, (int(w*1.8), int(h*1.8)))
	ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

	if median_kernel != None:
		thresh = cv2.medianBlur(thresh, median_kernel)

	if gaussian_kernel != None:
		thresh = cv2.GaussianBlur(thresh, (gaussian_kernel, gaussian_kernel), sigma)

	tmp_path = "./data/tmp/" + image_path[-9:]
	if method == 'dilation' and dilation_k != None:
		dilation = cv2.dilate(thresh, kernel_d, iterations=1)
		dilation_image = Image.fromarray(dilation, mode="L")
		# dilation_image.save(tmp_path,format='PNG')
		# dilation_buffer = io.BytesIO()
		# dilation_image.save(dilation_buffer,format='PNG')
		return dilation
	elif method == 'closing' and closing_k != None:
		closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel_c)
		closing_image = Image.fromarray(closing, mode="L")
		# closing_image.save(tmp_path,format='PNG')
		# closing_buffer = io.BytesIO()
		# closing_image.save(closing_buffer,format='PNG')
		return closing
	else:
		return thresh

	# cv2.imshow('Original', img)
	# # cv2.imshow('Blur', blur)
	# cv2.imshow('Median', median)
	# cv2.imshow('Dilation', dilation)
	# cv2.imshow('Closing', closing)

	# cv2.waitKey(0)
	# cv2.destroyAllWindows()

## Creating Images and Labels

In [12]:
def load_images_and_labels(data_dir, img_shape=(50, 200)):
    images = []
    labels = []
    # dilation or close
    method = "closing"

    # Listar todos os arquivos na pasta
    for file_name in os.listdir(data_dir):
        if file_name.endswith('.png') or file_name.endswith('.jpg'):
            # Caminho completo para a imagem
            img_path = os.path.join(data_dir, file_name)
            img = cv_image_processing(img_path,
                    gaussian_kernel=None, sigma=0.5,
                    median_kernel=None,
                    closing_k=(5,5),
                    dilation_k=(3,5),
                    method = method)
            # Carregar a imagem e redimensionar
            img = cv2.resize(img, img_shape)
            images.append(img)

            # Obter o rótulo (nome do arquivo sem a extensão)
            label = os.path.splitext(file_name)[0]
            labels.append(label)

    # Converter listas para arrays numpy
    images = np.array(images, dtype=np.float32) / 255.0  # Normalizar pixels (0 a 1)
    labels = np.array(labels)

    return images, labels

# Creating One-hot encoding for test

In [7]:
def encode_labels(labels, max_length=5):
    # Flatten os caracteres (separar cada caractere)
    chars = [char for label in labels for char in label]
    chars = np.array(chars).reshape(-1, 1)
    print(chars)
    # Criar o OneHotEncoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder.fit(chars)

    # Codificar cada rótulo
    encoded_labels = []
    for label in labels:
        # Codificar cada caractere e concatenar
        encoded = np.vstack([encoder.transform([[char]]) for char in label])
        encoded_labels.append(encoded)

    # Padronizar para comprimento fixo (max_length)
    encoded_labels = np.array(encoded_labels).reshape(-1, 5, 19)
    return encoded_labels, encoder


# Setting Leave one out with k-fold

In [22]:
def split_leave_one_out(images, labels, splits):
    loo = LeaveOneOut()
    folds =  KFold(n_splits=splits, shuffle=True, random_state=42)
    # for train_index, test_index in loo.split(images):
    #     train_images, test_images = images[train_index], images[test_index]
    #     train_labels, test_labels = labels[train_index], labels[test_index]

    #     folds.append((train_images, train_labels, test_images, test_labels))
    return folds

# Pipeline

In [9]:
# Diretório com os Captchas
data_dir = '../data/samples'

# 1. Carregar imagens e rótulos
images, labels = load_images_and_labels(data_dir, img_shape=(50, 200))

In [13]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)
train_labels, train_encoder = encode_labels(train_labels)
test_labels, test_encoder = encode_labels(test_labels)

model = build_crnn_with_attention(input_shape=(200,50, 1), num_classes=19)

# Compilar o modelo
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
# model.summary()

# Treinar o modelo
history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=50,  # Ajuste conforme necessário
    verbose=1
)



[['8']
 ['e']
 ['g']
 ...
 ['3']
 ['d']
 ['w']]
[['f']
 ['x']
 ['p']
 ...
 ['e']
 ['x']
 ['3']]
Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 36s 897ms/step - accuracy: 0.0570 - loss: 3.0456 - val_accuracy: 0.0953 - val_loss: 2.9263
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 728ms/step - accuracy: 0.1043 - loss: 2.9187 - val_accuracy: 0.0981 - val_loss: 2.8768
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 25s 928ms/step - accuracy: 0.1269 - loss: 2.8288 - val_accuracy: 0.1355 - val_loss: 2.7455
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 22s 814ms/step - accuracy: 0.1986 - loss: 2.5850 - val_accuracy: 0.1766 - val_loss: 2.5377
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 750ms/step - accuracy: 0.2837 - loss: 2.2785 - val_accuracy: 0.2439 - val_loss: 2.3657
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 762ms/step - accuracy: 0.3783 - loss: 1.9531 - val_accuracy: 0.2617 - val_loss: 2.2417
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 770ms/step - accuracy: 0.4396 - loss: 1.6868 - val_accuracy: 0.3112 - val_loss: 2.1155
E

In [14]:
loss, accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

Acurácia no conjunto de teste: 0.5888


In [131]:
encoded_labels, encoder = encode_labels(labels)

[['2']
 ['2']
 ['6']
 ...
 ['n']
 ['5']
 ['7']]


In [132]:
# # 3. Configurar Leave-One-Out Cross Validation
n_splits = 5
folds = split_leave_one_out(images, encoded_labels, splits=n_splits)

In [133]:
accuracy_per_fold = []
# Iterar sobre os folds
for fold, (train_idx, test_idx) in enumerate(folds.split(images)):
    print(f"Treinando Fold {fold + 1}/{n_splits}...")

    # Separar os dados do fold atual
    train_images, test_images = images[train_idx], images[test_idx]
    train_labels, test_labels = encoded_labels[train_idx], encoded_labels[test_idx]
    print(test_images.shape)
    # Criar um novo modelo para cada fold (para evitar reutilizar pesos)
    model = build_crnn_with_attention(input_shape=(200,50, 1), num_classes=19)

    # Compilar o modelo
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    # model.summary()

    # Treinar o modelo
    history = model.fit(
        train_images, train_labels,
        validation_data=(test_images, test_labels),
        batch_size=32,
        epochs=10,  # Ajuste conforme necessário
        verbose=1
    )

    # Avaliar no conjunto de teste
    loss, accuracy = model.evaluate(test_images, test_labels, verbose=0)
    print(f"Fold {fold + 1} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

    # Armazenar a acurácia do fold
    accuracy_per_fold.append(accuracy)

Treinando Fold 1/5...
(214, 200, 50)
Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 35s 918ms/step - accuracy: 0.0771 - loss: 2.9666 - val_accuracy: 0.0953 - val_loss: 2.9261
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 787ms/step - accuracy: 0.0991 - loss: 2.9174 - val_accuracy: 0.1056 - val_loss: 2.8598
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 25s 920ms/step - accuracy: 0.1421 - loss: 2.7909 - val_accuracy: 0.1710 - val_loss: 2.7136
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 23s 864ms/step - accuracy: 0.2196 - loss: 2.4906 - val_accuracy: 0.1981 - val_loss: 2.5089
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 799ms/step - accuracy: 0.3132 - loss: 2.1234 - val_accuracy: 0.2748 - val_loss: 2.2455
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 22s 795ms/step - accuracy: 0.4192 - loss: 1.7237 - val_accuracy: 0.2813 - val_loss: 2.2084
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 777ms/step - accuracy: 0.4759 - loss: 1.4643 - val_accuracy: 0.3280 - val_loss: 2.0772
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 23s 830ms/step - accur

In [135]:
# Resultados médios
print("\nResultados finais:")
print(f"Acurácia média: {np.mean(accuracy_per_fold):.4f}")
print(f"Desvio padrão da acurácia: {np.std(accuracy_per_fold):.4f}")


Resultados finais:
Acurácia média: 0.4054
Desvio padrão da acurácia: 0.0394


# Resultados 
## 5-fold 10 Epochs
### Dilation preprocessing:
- Acurácia média: 0.4105
- Desvio padrão da acurácia: 0.0174
### Closing preprocessing
- Acurácia média: 0.4054
- Desvio padrão da acurácia: 0.0394

## 0.2 Stratification
### 10 Epochs
#### Closing
- Acurácia média: 0.3916
#### Dilation: 
- Acurácia média: 0.3308

### 30 Epochs
#### Closing
- Acurácia média: 0
#### Dilation: 
- Acurácia média: 0.5355

### 50 Epochs
#### Closing
- Acurácia média: 0.
#### Dilation: 
- Acurácia média: 0.

### 100 Epochs
#### Closing
- Acurácia média: 0.
#### Dilation: 
- Acurácia média: 0.
